# Atividade 2: Modularidade e POO (Programação Orientada a Objetos)

## Objetivo
Demonstrar a aplicação de conceitos de **Engenharia de Código** e **Modularidade** em Python para isolar a lógica de negócio do fluxo de análise.

1.  **Encapsulamento POO:** Testar a criação e instanciação da classe `DataCleaner`, que encapsula dados e métodos (`calcular_faturamento`).
2.  **Modularidade:** Provar que a classe pode ser importada com sucesso do módulo `/src/datacleaner.py`.
3.  **Execução Isolada:** Utilizar um método da classe para adicionar a coluna `faturamento` ao dataset, mantendo o Notebook limpo.

Utilizaremos o mesmo dataset anterior, de **500.000 linhas**, para testar a classe de forma prática.

In [1]:
import pandas as pd
import sys

# Garantindo que o diretório /src está acessível para importação
sys.path.append('../src')

# Importando a classe DataCleaner
from datacleaner import DataCleaner

# Carregando o dataset
df = pd.read_csv('../data/df_produtos_500k.csv')

# Instanciando a classe
data_cleaner = DataCleaner(df)

#Aplicando o método
df_novo = data_cleaner.calcular_faturamento()

print("\nDataFrame com nova coluna 'faturamento':")
display(df_novo.head())

DataCleaner inicializado com 500000 linhas.

DataFrame com nova coluna 'faturamento':


,produto,preco_unitario,quantidade_vendida,categoria,faturamento
0,Produto_297,311.43,1467,Livros,456867.81
1,Produto_509,413.07,644,Livros,266017.08
2,Produto_587,46.87,1740,Alimentos,81553.80
3,Produto_546,45.28,135,Roupas,6112.80
4,Produto_693,294.53,1349,Eletronicos,397320.97


### Teste de Flexibilidade: Utilizando *kwargs no Construtor

Demonstração de como a classe `DataCleaner` aceita e armazena argumentos de configuração variáveis, preparando-a para receber parâmetros de filtragem ou estratégia em métodos futuros.

In [2]:
print("--- Testando Instanciação com Configurações Adicionais (kwargs) ---")

# Instanciando com os argumentos nomeados:
cleaner_flex = DataCleaner(
    df,
    estrategia_nulos='mediana',
    limite_max_preco=490.0,
    descricao="Pipeline de Limpeza - Vendas"
)

# Chamando o método
df_novo_flex = cleaner_flex.calcular_faturamento()

print("\nValidação: A mensagem acima deve mostrar o dicionário de configurações (**kwargs) impressas pelo __init__.")
display(df_novo_flex.tail(3))

--- Testando Instanciação com Configurações Adicionais (kwargs) ---
Configurações adicionais carregadas: {'estrategia_nulos': 'mediana', 'limite_max_preco': 490.0, 'descricao': 'Pipeline de Limpeza - Vendas'}
DataCleaner inicializado com 500000 linhas.

Validação: A mensagem acima deve mostrar o dicionário de configurações (**kwargs) impressas pelo __init__.


,produto,preco_unitario,quantidade_vendida,categoria,faturamento
499997,Produto_648,49.74,394,Roupas,19597.56
499998,Produto_964,299.97,1452,Alimentos,435556.44
499999,Produto_517,499.32,667,Servicos,333046.44


---

O objetivo de estabelecer uma arquitetura modular foi cumprido com sucesso.

* **Separação de Preocupações (SoC):** A lógica de negócio (`calcular_faturamento`) foi isolada do fluxo de análise, residindo integralmente no módulo `datacleaner.py` na pasta `/src`.
* **Fundação POO:** A classe `DataCleaner` demonstra o uso do **Encapsulamento**, gerenciando o estado do DataFrame internamente (`self.df`).
* **Extensibilidade:** A refatoração do `__init__` para aceitar `**kwargs` garante que a classe está pronta para receber configurações complexas (como limites de preços, estratégias de tratamento de nulos, etc.) em fases futuras, sem a necessidade de alterar sua interface (assinatura).


# Atividade 3: Pipeline e Decoradores

## Objetivo
Implementar um método de orquestração (`pipeline`) na classe `DataCleaner` e introduzir decoradores (`@classmethod` e `@staticmethod`) para otimizar o uso dos recursos da classe.

### Demonstração de Pipeline
A classe `DataCleaner` é refatorada para expor um método `pipeline()` que orquestra a execução sequencial de transformações. Isso garante que a ordem de processamento seja mantida e que o usuário execute todas as etapas de limpeza e preparação com uma única chamada, simulando um SDK de dados.

In [3]:
print("\n--- Testando o método pipeline() ---")

# Instanciando a classe

cleaner_pipeline = DataCleaner(
    df.copy(), # utilizando uma cópia do DF original para evitar alterações indesejadas
    estrategia_nulos='mediana',
    limite_max_preco=490.0,
    descricao="Pipeline de Limpeza - Vendas"
    )

# Chamando o método pipeline

df_pipeline = cleaner_pipeline.pipeline()

print('\nPipeline executado com sucesso. Dataframe resultante:')
display(df_pipeline.head())


--- Testando o método pipeline() ---
Configurações adicionais carregadas: {'estrategia_nulos': 'mediana', 'limite_max_preco': 490.0, 'descricao': 'Pipeline de Limpeza - Vendas'}
DataCleaner inicializado com 500000 linhas.
— Faturamento calculado.

Pipeline executado com sucesso. Dataframe resultante:


,produto,preco_unitario,quantidade_vendida,categoria,faturamento
0,Produto_297,311.43,1467,Livros,456867.81
1,Produto_509,413.07,644,Livros,266017.08
2,Produto_587,46.87,1740,Alimentos,81553.80
3,Produto_546,45.28,135,Roupas,6112.80
4,Produto_693,294.53,1349,Eletronicos,397320.97


### Recursos Avançados: Decoradores (`@classmethod` e `@staticmethod`)

A classe `DataCleaner` é aprimorada com Decoradores Python para demonstrar controle sobre a forma como seus métodos são acessados, aumentando a flexibilidade.

* **`@classmethod` (Fábrica de Objetos):** Cria um construtor alternativo (`from_csv`) que permite instanciar a classe diretamente a partir de um caminho de arquivo, eliminando a necessidade de carregar o DataFrame separadamente.
* **`@staticmethod` (Utilitário):** Adiciona uma função utilitária que é logicamente relacionada à classe (ex: formatação de números), mas que não depende do estado interno (o `self.df`).

In [4]:
print('\n --- Testando métodos com Decoradores (@classmethod e @staticmethod) ---')

print('\n --- 1. Testando o @classmethod ---')
# Instanciando a classe usando o método de fábrica from_csv, diretamente do arquivo CSV
cleaner_via_class = DataCleaner.from_csv(
    '../data/df_produtos_500k.csv',
    encoding = 'utf-8',
    estrategia_nulos = 'zeros'
)

# Executando o pipeline na nova instância criada
df_class_processado = cleaner_via_class.pipeline()
print('\nDataFrame processado via @classmethod from_csv:')
display(df_class_processado.head())

print('\n--- 2. Testando o @staticmethod ---')
# Utilizando o método estático para formatar um valor, sem precisar instanciar a classe

valor_exemplo = 247198.45
valor_formatado = DataCleaner.converter_para_milhares(valor_exemplo)

print(f'\nValor original: R$ {valor_exemplo}')
print(f'\nValor formatado via @staticmethod (em milhares): {valor_formatado}k')


 --- Testando métodos com Decoradores (@classmethod e @staticmethod) ---

 --- 1. Testando o @classmethod ---
Configurações adicionais carregadas: {'estrategia_nulos': 'zeros'}
DataCleaner inicializado com 500000 linhas.
— Faturamento calculado.

DataFrame processado via @classmethod from_csv:


,produto,preco_unitario,quantidade_vendida,categoria,faturamento
0,Produto_297,311.43,1467,Livros,456867.81
1,Produto_509,413.07,644,Livros,266017.08
2,Produto_587,46.87,1740,Alimentos,81553.80
3,Produto_546,45.28,135,Roupas,6112.80
4,Produto_693,294.53,1349,Eletronicos,397320.97



--- 2. Testando o @staticmethod ---

Valor original: R$ 247198.45

Valor formatado via @staticmethod (em milhares): 247.198k


---

Com a implementação do pipeline e dos decoradores, a classe `DataCleaner` está agora completa como uma API de processamento de dados robusta e aderente às melhores práticas de Engenharia de Software.

* **Orquestração de Pipeline:** O método `pipeline()` centraliza e garante a ordem correta das transformações, promovendo a **Manutenibilidade** e simplificando o uso para o cientista de dados.
* **Controle de Acesso (`@classmethod`):** O uso do `@classmethod` demonstra o domínio de construtores alternativos (`from_csv`), tornando a inicialização da classe mais prática e flexível, alinhada ao design de SDKs profissionais.
* **Separação de Funções (`@staticmethod`):** O `@staticmethod` isola funções utilitárias que não dependem do estado da classe, melhorando a organização lógica.

O projeto está agora pronto para a fase final: a documentação completa.